In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plot
plots.style.use('fivethirtyeight')

## Lecture 6

## Table Review: Sleep Study

In [ ]:
sleep = Table.read_table('SleepStudy.csv')
sleep

**Question:** I will like to do some analysis on only fourth year students. Return a table of only 4th year students.

In [ ]:
sleep.where('ClassYear', 4)

## Introduction to Predicates for Querying Rows of a Table ##

**Question:** Can you use one appropriate `datascience` to return the last table above instead too? Boht answers are correct. But sometimes, these predicates can be handy.

In [ ]:
sleep.where('ClassYear', are.equal_to(4))

**Question:** Return a table with students who average 7 or more, but less than 9 hours each night.

In [ ]:
sleep.where('AverageSleep', are.between(7,9))

**Question:** Return a table with students who average between 7 and 9 hours of sleep each night.

In [ ]:
sleep.where('AverageSleep', are.strictly_between(7,9))

**Question:** On average, how long does a "Lark" student sleep?

In [ ]:
Larks= sleep.where('LarkOwl', 'Lark')
np.average(Larks.column('AverageSleep'))

**Question:** Use predicates to return the same result below.

In [ ]:
Larks= sleep.where('LarkOwl', are.containing('Lark'))
np.average(Larks.column('AverageSleep'))

**Question:** Suppose students at risk of failing have a GPA of 2.50 or below. What is the maximum number of alcoholic drinks taken per week by a student at risk of failing?

In [ ]:
at_risk = sleep.where('GPA', are.below_or_equal_to(2.50))
max(at_risk.column('Drinks'))

**Question:** How many students missed fewer than 3 classes?

In [ ]:
sleep.where('ClassesMissed', are.below(3)).num_rows

**Question:** What proportion of students do not abstain from alcohol use?

In [ ]:
num_alcohol_use = sleep.where('AlcoholUse', are.not_containing('Abstain')).num_rows
num_alcohol_use / sleep.num_rows

[Click here to read more about predicates available in the datascience package.](https://www.data8.org/datascience/predicates.html)

## Census ##

In [ ]:
full = Table.read_table('nc-est2014-agesex-res.csv')
full

In [ ]:
partial = full.select('SEX', 'AGE', 'CENSUS2010POP', 'POPESTIMATE2014')
partial

**Question:** Suppose I would like to relabel the third and fourth columns as just `2010` and `2014`. How can I do this?

In [ ]:
simple = partial.relabel(2, '2010').relabel(3, '2014')
simple

## Checking Data Integrity Issues ##

**Question:** The accompanying meta data for this data does indicate some columns like `SEX` and `AGE` has interesting data entries. Should a good data scientist check this out before conducting analysis using the data set?

In [ ]:
simple.sort('AGE')

In [ ]:
simple.sort('AGE', descending=True)

## Visualization ##

Tables are a powerful way of organizing and visualizing data. However, large tables of numbers can be difficult to interpret, no matter how organized they are. Sometimes it is much easier to interpret graphs than numbers.

**Question:** Provide a plot of census population count in 2010 by the different ages. 

- what can you infer from the graph? 
- Does it reflect reality?

In [ ]:
no_999 = simple.where('AGE', are.below(999))
everyone = no_999.where('SEX', 0).drop('SEX')

In [ ]:
everyone

In [ ]:
everyone.plot('AGE', '2010') #.plot(variable on the x-axis, variable on the y-axis)

**Question:** What proportion of each age group were female in 2010?

In [ ]:
no_999

In [ ]:
no_999.group('SEX')

In [ ]:
uspop_mf_2010 = no_999.select('AGE','2010', 'SEX').where('SEX', are.above(0))
uspop_mf_2010

In [ ]:
females_uspop_2010 = uspop_mf_2010.where('SEX', are.equal_to(2)).drop('SEX').relabeled(1, 'females_2010')
males_uspop_2010 = uspop_mf_2010.where('SEX', are.equal_to(1)).drop('SEX').relabeled(1, 'males_2010')
females_uspop_2010.take(np.arange(5))

In [ ]:
males_uspop_2010.take(np.arange(5))

In [ ]:
prop_female_uspop_2010 =  Table().with_column('Proportion Female', 
                                            females_uspop_2010.column('females_2010') / 
                                              (females_uspop_2010.column('females_2010') + males_uspop_2010.column('males_2010')))
prop_female_uspop_2010

**Question:** Create a table having the last column above with proportion of females in each age group and AGE?

In [ ]:
prop_female_uspop_2010 = prop_female_uspop_2010.with_column('AGE', females_uspop_2010.column('AGE'))
prop_female_uspop_2010

**Question:** Plot the proportion of females in each age group in 2010 by age?

In [ ]:
prop_female_uspop_2010.plot('AGE', 'Proportion Female')